In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get -y install protobuf-compiler python-pil python-lxml

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-pil
E: Unable to locate package python-lxml


In [ ]:
!pip install openNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.w

In [ ]:
config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Tsonga_Tsonga_Parralel_train.txt.subword
        path_tgt: /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Tsonga_English_Parralel_train.txt.subword
        transforms: [filtertoolong]
    valid:
        path_src: /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Tsonga_Tsonga_dev.txt.subword
        path_tgt: /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Tsonga_English_dev.txt.subword
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 64
tgt_seq_length: 64

# Tokenization options
src_subword_model: /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/source.model
tgt_subword_model: /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/target.model
# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.tsongaeng

# Stop training if it does not imporve after n validations
early_stopping: 10

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 2023

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 6000

# Default: 10000 - Run validation after n steps
valid_steps: 4000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 5000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 1024  # Tokens per batch, change when CUDA out of memory
valid_batch_size: 1024
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 0.4
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 8
dec_layers: 8
heads: 8
hidden_size: 1024
word_vec_size: 1024
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
!onmt_build_vocab -config /content/config.yaml -n_sample -1 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-08-08 09:13:00,414 INFO] Counter vocab from -1 samples.
[2024-08-08 09:13:00,414 INFO] n_sample=-1: Build vocab on full datasets.
[2024-08-08 09:13:05,173 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=708)

[2024-08-08 09:13:05,189 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=683)

[2024-08-08 09:13:05,253 INFO] Counters src: 30033
[2024-08-08 09:13:05,253 INFO] Counters tgt: 26931


In [ ]:
 # Train the NMT model
!onmt_train -config /content/config.yaml

[2024-08-08 09:13:08,856 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-08-08 09:13:08,856 INFO] Parsed 2 corpora from -data.
[2024-08-08 09:13:08,856 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-08-08 09:13:09,088 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '▁ku', '▁.', '▁ya', '▁na', '▁wa', '▁hi']
[2024-08-08 09:13:09,088 INFO] The decoder start token is: <s>
[2024-08-08 09:13:09,088 INFO] Building model...
[2024-08-08 09:13:12,030 INFO] Switching model to float32 for amp/apex_amp
[2024-08-08 09:13:12,030 INFO] Non quantized layer compute is fp16
[2024-08-08 09:13:12,429 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(30040, 1024, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transfor

In [ ]:
!onmt_translate -model /content/models/model.tsongaeng_step_6000.pt -src /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Tsonga_Tsonga_test.txt.subword -output /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Ukuxhumana8heads.Translated

[2024-08-08 09:46:36,151 INFO] Loading checkpoint from /content/models/model.tsongaeng_step_6000.pt
[2024-08-08 09:46:38,529 INFO] Loading data into the model
[2024-08-08 09:50:06,643 INFO] PRED SCORE: -0.2737, PRED PPL: 1.31 NB SENTENCES: 3000
Time w/o python interpreter load/terminate:  210.59305357933044


In [ ]:
!mkdir -p nmt
%cd nmt
!git clone https://github.com/ymoslem/MT-Preparation.git

/content/nmt
Cloning into 'MT-Preparation'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 290 (delta 145), reused 189 (delta 97), pack-reused 0
Receiving objects: 100% (290/290), 75.16 KiB | 1.47 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [ ]:
# Install the requirements
!pip3 install -r MT-Preparation/requirements.txt

In [ ]:
%cd  ../

/content


In [ ]:
!ls

config.yaml  drive  models  nmt  run  sample_data  train.log


In [ ]:
!python3 nmt/MT-Preparation/subwording/3-desubword.py /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/target.model /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Ukuxhumana8heads.Translated

Done desubwording! Output: /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Ukuxhumana8heads.Translated.desubword


In [ ]:
!python3 nmt/MT-Preparation/subwording/3-desubword.py /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/target.model /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Tsonga_English_test.txt.subword

Done desubwording! Output: /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Tsonga_English_test.txt.subword.desubword


In [ ]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
!pip3 install sacrebleu

--2024-08-08 09:50:11--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2024-08-08 09:50:11 (87.2 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [ ]:
!python3 compute-bleu.py /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Ukuxhumana8heads.Translated.desubword  /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Tsonga_English_test.txt.subword.desubword

Reference 1st sentence: is the subject of children who are still children .
MTed 1st sentence: on the subject of child soldiers .
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU:  61.065151685234504


In [ ]:
!onmt_translate -model /content/models/model.tsongaeng_step_6000.pt -src /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/CCAligned.en-ts.txt.subword  -output /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/SyntheticEnglish.Translated

[2024-08-08 09:50:17,366 INFO] Loading checkpoint from /content/models/model.tsongaeng_step_6000.pt
[2024-08-08 09:50:19,952 INFO] Loading data into the model
[2024-08-08 09:55:35,846 INFO] PRED SCORE: -0.7940, PRED PPL: 2.21 NB SENTENCES: 1967
Time w/o python interpreter load/terminate:  318.78488302230835


In [ ]:
!python3 nmt/MT-Preparation/subwording/3-desubword.py /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/target.model /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/SyntheticEnglish.Translated

Done desubwording! Output: /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/SyntheticEnglish.Translated.desubword


In [ ]:
!wc -l /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/SyntheticEnglish.Translated
!wc -l /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/CCAligned.en-ts-en.txt.subword

1967 /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/SyntheticEnglish.Translated
1968 /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/CCAligned.en-ts-en.txt.subword


In [ ]:
!python3 compute-bleu.py /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/SyntheticEnglish.Translated /content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/CCAligned.en-ts-en.txt.subword

Reference 1st sentence: ▁To ▁ 1 ▁m .
MTed 1st sentence: ▁Up ▁to ▁ 1 ▁m .
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU:  15.82008978978491


# METEOR Score Computation

In [ ]:
from nltk.translate.meteor_score import meteor_score, single_meteor_score
import nltk

In [ ]:
# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('punkt')

# Function to read the content of a text file
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Tokenize the text
def tokenize(text):
    return nltk.word_tokenize(text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Function to read the content of a text file
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Paths to your text files
file1_path = '/content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Ukuxhumana8heads.Translated.desubword'
file2_path = '/content/drive/MyDrive/LanguageTranslationData_New/BackTranslation/TsongaEng/English_Tsonga_English_test.txt.subword.desubword'

# Read the contents of the files
reference = read_file(file1_path)
hypothesis = read_file(file2_path)

# Tokenize the texts
reference_tokens = tokenize(reference)
hypothesis_tokens = tokenize(hypothesis)

# Calculate the METEOR score
score = meteor_score([reference_tokens], hypothesis_tokens)

print(f'The METEOR score between the two files is: {score}')

The METEOR score between the two files is: 0.5650550587374646
